Задача: вытащить из соответствующего файла базы ФИАС:

1) улицы с нужным названием, добавить к ним регион, м. о., город или населенный пункт;

2) города и населенные пункты

3) муниципальные образования с нужной семантикой и регионом

Регион добавляем из отдельного файла, м.о. и город -- по расщепленному коду ФИАС (plaincode).

Это отладочный код, на примере Московской области, одной из самых полноводных.

Отладочный код в -- 01_FIAS_street_parsing

Код вытаскивает и записывает в файлы данные из ФИАС.

Всего 6 папок, в каждой по 86 файлов с одинаковыми именами (01_Адыгея.csv и т.п.)

Файл reg_code.scv -- https://drive.google.com/file/d/1VlFxl333anKArVC0nsevpr1_S0vs5qjz/

In [1]:
import os
from tqdm import tqdm_notebook

import pandas as pd
import numpy as np

from dbfread import DBF

In [2]:
# настройки панды для отображения ОКТМО не в экспоненциальной записи

pd.set_option('precision', 13)

In [3]:
# загружаем файл с кодами регионов (чтобы потом присваивать файлам нужные имена)

reg_code_new = pd.read_csv('C:/00_Projects/Lenin/Datasets/03_Other/reg_code.csv', index_col='regioncode')

In [4]:
# создаем датафрейм с названиями регионов и кодами ФИАС

reg_code = pd.read_csv('C:/00_Projects/Lenin/Datasets/03_Other/reg_code.csv')
reg_code.regioncode = reg_code.regioncode.astype('int32')

In [5]:
# создаем на основе списка регионов словарь (чтобы потом присваивать файлам нужные имена)

region_name = reg_code_new.to_dict('index')

In [6]:
# вытаскиваем названия таблиц в список file_list

file_list = []
for root, dirs, files in os.walk('C:/00_Projects/Lenin/FIAS'):
    for filename in files:
        if 'ADDROB' in filename:
            file_list += [filename]

In [7]:
# Функция для разметки данных (нужна в цикле)
# Режет plaincode в соответствии с "муниципальным статусом" объекта

def cut_code(row):
    
    # муниципалитеты
    
    if row.aolevel == 3: 
        return row.plaincode[0:5]
    
    # городские округа (у г.о. areacode - 000)
    
    elif row.aolevel == 4:
        if row.areacode == '000':
            return row.plaincode[0:8]
        
        else:
            return row.plaincode[0:5]

    # поселки и улицы
    
    elif row.aolevel == 6 or row.aolevel == 7:
        
        if row.areacode != '000':
            return row.plaincode[0:5]
                
        # поселки которые имеют статус округов
        
        elif row.areacode == '000' and row.citycode == '000':
            return row.plaincode[0:11]            
        
        # все остальное (000 в areacode означает, что объект входит в состав г.о. или с.о.)
        # поэтому берем 8 цифр
        
        else:
            return row.plaincode[0:8]

In [ ]:
# примитивная функция для заполнения по ОКТМО (смотрит на другие строки с таким же ОКТМО)
# для городов регионов (Москвы, СПб, Севастополя) функция вместо м.о. добавляет населенные пункты

def add_mun_district(row):
    need_code = row.oktmo
    try:
        new_value = work[(
            (work.oktmo == need_code) &
            (work.mun_district != 'null')
        )].mun_district.to_list()[0]
        
    except:
        new_value = 'null'
        
    return new_value

In [9]:
%%time

# вытаскиваем циклом данные из ФИАСа
for i in tqdm_notebook(file_list):
    
    
    # читаем файл
    dbf = DBF('C:/00_Projects/Lenin/FIAS/' +i)
    data = pd.DataFrame(dbf)
    
    
    # фильтруем данные, оставляем только актуальные адреса LIVESTATUS = 1
    data = data[data.LIVESTATUS == 1]
    
    
    # удаляем дубликаты
    data = data.drop_duplicates()
    
    
    
    
    
    # рабочий датафрейм
    
    
    # отбираем необходимые столбцы для рабочего датафрейма
    necessary_columns = ['AOLEVEL', 'FORMALNAME', 'OFFNAME', 'SHORTNAME',
                             'PLAINCODE', 'REGIONCODE', 'AREACODE', 'CITYCODE',
                             'OKATO', 'OKTMO', 'POSTALCODE']    
    
    
    # оставляем только нужные столбцы в новом фрейме work
    work = data[necessary_columns]
    work.columns = work.columns.str.lower()
    
    
    # меняем типы столбцов для корректной работы
    work = work.astype({'aolevel': 'int32', 'regioncode': 'int32'})
    
    
    # оставляем только нужные записи
    work = (work[
        (work.aolevel == 1) |
        (work.aolevel == 3) |
        (work.aolevel == 4) |
        (work.aolevel == 6) |
        (work.aolevel == 7) ])
    
    
    
    
    # добавляем регион
    
    
    # файл с кодами уже загружен    
    work = work.merge(reg_code, on='regioncode', how='left')
    
    
    
    
    
    
    # Добавляем муниципальный район
        
        
    # отбираем районы и города и поселки регионального подчинения
    mun_district = work[(
        (work.aolevel == 3) |
        (
            (work.aolevel == 4) &
            (work.areacode == '000' )) |
        (
            (work.aolevel == 6) &
            (work.areacode == '000') &
            (work.citycode == '000')))]
    
    
    # добавляем колонку с кодом региона и района
    mun_district['mun_code'] = mun_district.apply(cut_code, axis=1)
    
    
    # оставляем нужные столбцы
    mun_district = mun_district[['formalname', 'mun_code']]
    
    
    # переименовываем
    mun_district = mun_district.rename(columns={'formalname': 'mun_district'})
    
    
    # добавляем mun_code в датафрейм
    work['mun_code'] = work.apply(cut_code, axis=1)
    
    
    # объединяем, т.е. добавляем к улице м.о.
    work = work.merge(mun_district, on='mun_code', how='left')
    
    
    
    
    
    
    #Пропуски в данных
    
    
    # заполняем пустые значения
    #work.fillna('null', inplace=True)
    
    
    # создаем временный датафрейм
    #temp_md = work[work.mun_district == 'null']
    
    
    # добавляем столбец
    #temp_md['new_md'] = temp_md.apply(add_mun_district, axis=1)
    
    
    # оставляем нужные столбцы для слияния (на всякий случай 4)
    #temp_md = temp_md[['formalname', 'offname', 'plaincode', 'oktmo', 'new_md']]
    
    
    # соединяем
    #work = work.merge(temp_md, on=['formalname', 'offname', 'plaincode', 'oktmo'], how='left')
    
    
    # заполняем NaN
    #work.fillna('null', inplace=True)
    
    
    # меняем null в mun_district
    #work.loc[(work.mun_district == 'null'), 'mun_district'] = work.new_md
    
    
    
    
        
        
    #Добавляем населенный пункт    
    
    
    # создаем датафрейм с городами и поселками
    city = work[(
        (work.aolevel == 4) |
        (work.aolevel == 6))]
    
    
    # оставляем нужные столбцы
    city = city[['shortname', 'formalname', 'plaincode']]
    
    
    # переименовываем
    city = city.rename(columns={'shortname': 'city_type', 'formalname': 'city', 'plaincode': 'city_code'})
    
    
    # создаем столбец "код города" для слияния
    work['city_code'] = work.plaincode.str[0:11]
    
    
    # объединяем
    work = work.merge(city, on='city_code', how='left')
    
    
    # заполняем пустые значения
    work.fillna('null', inplace=True)
    
    
    # берем city из mun_district
    work.loc[(work.city == 'null'), 'city'] = work.mun_district
    
    
    
    
    
    # Фильтруем данные
    
        
    # переименовываем formalname в понятный street
    work.rename(columns={'formalname': 'street'}, inplace=True)
            
    
    
    # фильтруем по нужной семантике
    lenin_work = work[(
    (work.street.str.contains('ленин', case=False)) |
    (work.street.str.contains('ульянов', case=False)) |
    (work.street.str.contains('ильич', case=False)))]
    
    
    # датафрейм муниципальных образований
    work_districts = work[(
        (work.aolevel == 3) |
        (
            (work.aolevel == 4) &
            (work.areacode == '000')) |
        (
            (work.aolevel == 6) &
            (work.areacode == '000') &
            (work.citycode == '000')))]
    
    
    # оставляем важные столбцы
    work_districts = work_districts[['aolevel', 'shortname', 'mun_district', 'offname',
                                     'region',
                                     'okato', 'oktmo']]
    
    
    #  "ленинские" муниципальные районы и города регионального подчинения
    lenin_districts = lenin_work[(
        (lenin_work.aolevel == 3) |
        (
            (lenin_work.aolevel == 4) &
            (lenin_work.areacode == '000')) |
        (
            (lenin_work.aolevel == 6) &
            (lenin_work.areacode == '000') &
            (lenin_work.citycode == '000')))]
    
        
    # оставляем важные столбцы
    lenin_districts = lenin_districts[['aolevel', 'shortname', 'mun_district', 'offname',
                                       'region',
                                       'okato', 'oktmo']]
    
    
    #  фильтруем города и поселки
    work_cities = work[(
        (work.aolevel == 4) |
        (work.aolevel == 6) |
     (
        (work.aolevel == 3) &
        ((work.shortname == 'г') |
         (work.shortname == 'п'))
    )
    )]
    
    
    # оставляем важные столбцы
    work_cities = work_cities[['aolevel', 'city_type', 'city', 'offname',
                           'mun_district', 'region',
                           'okato', 'oktmo', 'postalcode']]

    
    
    #  фильтруем "ленинские" города и поселки
    lenin_cities = lenin_work[(
        (lenin_work.aolevel == 4) |
        (lenin_work.aolevel == 6)
    )]
    
    
    # оставляем важные столбцы
    lenin_cities = lenin_cities[['aolevel', 'city_type', 'city', 'offname',
                                 'mun_district', 'region',
                                 'okato', 'oktmo', 'postalcode']]
    
    
    #  фильтруем улицы и территории
    work_streets = work[work.aolevel == 7]
    
    
    # оставляем важные столбцы
    work_streets = work_streets[['aolevel', 'shortname', 'street', 'offname',
                                 'city_type', 'city', 'mun_district', 'region',
                                 'okato', 'oktmo', 'postalcode']]
    
    
    #  фильтруем "ленинские" улицы и территории
    lenin_streets = lenin_work[lenin_work.aolevel == 7]
    
    
    # оставляем важные столбцы
    lenin_streets = lenin_streets[['aolevel', 'shortname', 'street', 'offname',
                                   'city_type', 'city', 'mun_district', 'region',
                                   'okato', 'oktmo', 'postalcode']]
    
    
    
    
    
    # записываем в файлы
    
        
    # Имена + адреса
    csv_name_01 = 'C:/00_Projects/Lenin/00_FIAS_pars/01_all_districts/' +(i[6:8]) +'_' +region_name[int(i[6:8])]['region'] +'.csv'
    csv_name_02 = 'C:/00_Projects/Lenin/00_FIAS_pars/02_lenin_districts/' +(i[6:8]) +'_' +region_name[int(i[6:8])]['region'] +'.csv'
    csv_name_03 = 'C:/00_Projects/Lenin/00_FIAS_pars/03_all_cities/' +(i[6:8]) +'_' +region_name[int(i[6:8])]['region'] +'.csv'
    csv_name_04 = 'C:/00_Projects/Lenin/00_FIAS_pars/04_lenin_cities/' +(i[6:8]) +'_' +region_name[int(i[6:8])]['region'] +'.csv'
    csv_name_05 = 'C:/00_Projects/Lenin/00_FIAS_pars/05_all_streets/' +(i[6:8]) +'_' +region_name[int(i[6:8])]['region'] +'.csv'
    csv_name_06 = 'C:/00_Projects/Lenin/00_FIAS_pars/06_lenin_streets/' +(i[6:8]) +'_' +region_name[int(i[6:8])]['region'] +'.csv'
    
    
    # запись
    work_districts.to_csv(csv_name_01, sep=',', encoding='utf-8', index=False)
    lenin_districts.to_csv(csv_name_02, sep=',', encoding='utf-8', index=False)
    work_cities.to_csv(csv_name_03, sep=',', encoding='utf-8', index=False)
    lenin_cities.to_csv(csv_name_04, sep=',', encoding='utf-8', index=False)
    work_streets.to_csv(csv_name_05, sep=',', encoding='utf-8', index=False)
    lenin_streets.to_csv(csv_name_06, sep=',', encoding='utf-8', index=False)
    
    
    #break

C:\Users\Boris\Anaconda3\lib\site-packages\ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Boris\Anaconda3\lib\site-packages\ipykernel_launcher.py:112: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Wall time: 20min 11s


### Объединяем "региональные" csv в общие ###

Байконур не включаем.

In [10]:
# создаем список файлов, названия файлов во всех папках одинаковы, поэтому список один

csv_list = os.listdir('C:/00_Projects/Lenin/00_FIAS_pars/01_all_districts/')

In [11]:
# создаем список папок

list_folders = list(os.walk('C:/00_Projects/Lenin/00_FIAS_pars/'))[0][1]

In [12]:
list_folders

['01_all_districts',
 '02_lenin_districts',
 '03_all_cities',
 '04_lenin_cities',
 '05_all_streets',
 '06_lenin_streets']

In [ ]:
%%time

# записываем файлы в csv

# цикл по папкам
for folder in tqdm_notebook(list_folders):
    url = 'C:/00_Projects/Lenin/00_FIAS_pars/' + folder + '/'

    # создаем временный датафрейм раздела
    df_temp_1 = pd.read_csv(url + csv_list[0])
    
    
    # цикл по файлам (Байконур не включаю, поэтому range до 85)
    for i in range(1, 85):
        
        # записываем файлы во второй датафрейм и присоединяем к первому        
        df_temp_2 = pd.read_csv(url + str(csv_list[i]))
        df_temp_1 = df_temp_1.append(df_temp_2, ignore_index = True)
        
    # записываем фрейм в файл:
    df_temp_1.to_csv('C:/00_Projects/Lenin/01_FIAS_csv_original/' + str(folder) +'.csv', sep=',', encoding='utf-8', index=False)

C:\Users\Boris\Anaconda3\lib\site-packages\pandas\core\frame.py:7123: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,
